In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from ipywidgets import widgets
from cec2017.functions import *

In [2]:
def boundery_checker(pop, min_val, max_val):
    adj_pop = np.where(pop<min_val, min_val, pop)
    adj_pop = np.where(adj_pop>max_val, max_val, adj_pop)
    return adj_pop

def init_genrator_tlbo(min_val, max_val, num_variables, pop_size):
    return np.random.uniform(min_val, max_val, (pop_size, num_variables))

def sorting_pop(pop, obj_func):
    results = obj_func(pop)
    indeces = np.argsort(results)
    return pop[indeces]

def teacher_phase(pop, pop_size, obj_func, min_val, max_val):
    teacher = pop[0]
    center = np.mean(pop, axis=0)
    rand = np.random.uniform(0,1,size=(pop_size,1))
    teacher_factor = np.random.choice(np.arange(1,3), size=(pop_size,1))
    diff = rand*(teacher-center*teacher_factor)
    pop_new = pop + diff
    pop_new = boundery_checker(pop_new, min_val, max_val)
    results = obj_func(pop)
    results_new = obj_func(pop_new)
    return np.where((results_new<results).reshape(-1,1), pop_new, pop)

def learner_phase(pop, pop_size, obj_func, min_val, max_val):
    rand = np.random.uniform(0,1,size=(pop_size,1))
    pop_new = np.zeros_like(pop)
    for i in range(pop_size):
        x, y = np.sort(np.random.choice(np.arange(pop_size), size=2, replace=False))
        pop_new[i] = pop[i] + rand[i]*(pop[x]-pop[y])
    pop_new = boundery_checker(pop_new, min_val, max_val)
    results = obj_func(pop)
    results_new = obj_func(pop_new)
    return np.where((results_new<results).reshape(-1,1), pop_new, pop)

def tlbo_sep(pop_size, iteration, min_val, max_val, num_variables, obj_func):
    NFE_value = 0
    NFE = np.zeros(iteration)
    result = np.zeros(iteration)
    pop = init_genrator_tlbo(min_val, max_val, num_variables, pop_size)
    pop = sorting_pop(pop, obj_func)
    for i in range(iteration):
        pop = teacher_phase(pop, pop_size, obj_func, min_val, max_val)
        pop = sorting_pop(pop, obj_func)
        pop = learner_phase(pop, pop_size, obj_func, min_val, max_val)
        pop = sorting_pop(pop, obj_func)
        NFE_value += (3*pop_size)
        result[i] = obj_func([pop[0]])
        NFE[i] = NFE_value
    return pop[0], obj_func([pop[0]]), result, NFE

In [3]:
def opt_plotting(obj_func_name, num_variables=10, min_val=-100, max_val=100, pop_size=20, iteration=700):
    obj_func = eval(obj_func_name)
    X_best, OF_best, result, NFE = tlbo_sep(pop_size, iteration, min_val, max_val, num_variables, obj_func)
    fig = plt.plot(NFE, result)
    fig = plt.grid(alpha=.5)
    fig = plt.margins(0,.05)
    fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')
    fig = plt.title('Optimum point: %.2E'%OF_best)
    plt.tight_layout()

funct_name = ['f%d'%i for i in range(1, 31)]

In [4]:
widgets.interact(opt_plotting, obj_func_name=funct_name, num_variables=[2, 10, 50], min_val=(-100, -10, 10), 
                 max_val=(10, 100, 10), pop_size=(10, 100, 10), iteration=(100, 5000, 100))

interactive(children=(Dropdown(description='obj_func_name', options=('f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7',…

<function __main__.opt_plotting(obj_func_name, num_variables=10, min_val=-100, max_val=100, pop_size=20, iteration=700)>

<div class='alert alert-info'>
    <h5>2D space represantation</h5>
</div>

In [5]:
def plot_2D(obj_func_name, min_val=-100, max_val=100, pop_size=20, iteration=700, xgap=.07, ygap=.04,
           mesh_size=200, color_map1='viridis', color_map2 = 'viridis'):
    num_variables=2
    obj_func = eval(obj_func_name)
    X_best, OF_best, result, NFE = tlbo_sep(pop_size, iteration, min_val, max_val, num_variables, obj_func)
    
    
    fig = plt.figure(figsize=(8, 3))
    fig = plt.suptitle('Optimum point: %.2E'%OF_best)
    plt.subplot(1, 2, 1)
    x1_full = np.linspace(min_val, max_val, 100)
    x2_full = np.linspace(min_val, max_val, 100)
    X1, X2 = np.meshgrid(x1_full, x2_full)
    flat_mesh = np.c_[X1.flat, X2.flat]
    Z = obj_func(flat_mesh).reshape(X1.shape)
    fig = plt.contourf(X1, X2, Z, extend='max', cmap = color_map1)
    cbar = plt.colorbar(fig)
    fig = plt.scatter(X_best[0], X_best[1], marker='x', c='r', s=15)

    plt.subplot(1, 2, 2)
    obj_func = eval(obj_func_name)
    X_best, OF_best, result, NFE = tlbo_sep(pop_size, iteration, min_val, max_val, num_variables, obj_func)
    x1_min, x1_max = np.array([1-xgap, 1+xgap])*X_best[0]
    x2_min, x2_max = np.array([1-ygap, 1+ygap])*X_best[1]
    x1 = np.linspace(x1_min, x1_max, mesh_size)
    x2 = np.linspace(x2_min, x2_max, mesh_size)
    X1, X2 = np.meshgrid(x1, x2)
    flat_mesh = np.c_[X1.flat, X2.flat]
    Z = obj_func(flat_mesh).reshape(X1.shape)
    fig = plt.contourf(X1, X2, Z, extend='max', cmap=color_map2)
    cbar = plt.colorbar(fig)
    fig = plt.scatter(X_best[0], X_best[1], marker='x', c='r', s=15)

    plt.tight_layout()
    
cm_names = ['viridis', 'plasma', 'inferno', 'magma', 'cividis',
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges',
            'Reds', 'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu',
            'BuPu','GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn',
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper', 'PiYG', 'PRGn', 'BrBG',
            'PuOr', 'RdGy', 'RdBu', 'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm',
            'bwr', 'seismic', 'twilight', 'twilight_shifted', 'hsv']
obj_func_list = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10']

In [6]:
widgets.interact(plot_2D, obj_func_name=obj_func_list, min_val=(-100, -10, 20), max_val=(10, 101, 20),
                 pop_size=(10, 100, 20), iteration=(100, 2000, 200), mesh_size=[100, 200, 500], xgap=(.01, 1.5, .01),
                 ygap=(.01, 1.5, .01), color_map1=cm_names, color_map2 = cm_names)

interactive(children=(Dropdown(description='obj_func_name', options=('f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7',…

<function __main__.plot_2D(obj_func_name, min_val=-100, max_val=100, pop_size=20, iteration=700, xgap=0.07, ygap=0.04, mesh_size=200, color_map1='viridis', color_map2='viridis')>